In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# 한 셀에서 여려 결과 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [13]:
# stack : 여러 열을 하나의 열로 쌓아 올려 데이터 구조를 변경
#         다중 인덱스를 생성하게 됨
#         결측치 제거를 해줌

s = df1.stack()
# s.index
r = df1.set_index(["도시", "연도"])
r
r.unstack(level = "도시")

인구   지역
도시 연도                
서울 2015  9904312  수도권
   2010  9631482  수도권
   2005  9762546  수도권
부산 2015  3448737  경상권
   2010  3393191  경상권
   2005  3512547  경상권
인천 2015  2890451  수도권
   2010   263203  수도권

인구                         지역          
도시           부산         서울         인천   부산   서울   인천
연도                                                  
2005  3512547.0  9762546.0        NaN  경상권  수도권  NaN
2010  3393191.0  9631482.0   263203.0  경상권  수도권  수도권
2015  3448737.0  9904312.0  2890451.0  경상권  수도권  수도권

In [17]:
df = pd.read_json("./data/fakedata.json")
df = pd.json_normalize(df["items"].iat[0]) # = df["items"][0]
df = df[["occrrnc_dt", "dght_cd", "dth_dnv_cnt", "injpsn_cnt"]]

# 월별 사망자수와 부상자수 합계를 확인
df["date"] = pd.to_datetime(df["occrrnc_dt"].str[0:8])
# df_1 = df.groupby(df["date"].dt.month)[["dth_dnv_cnt", "injpsn_cnt"]].sum()
# df_1.plot()

# 요일별 낮밤별 사망자 부상자 수 구하기
t = df.groupby([df["date"].dt.dayofweek, "dght_cd"])[["dth_dnv_cnt", "injpsn_cnt"]].sum()
t = t.unstack(level = "date")

In [18]:
fig= px.imshow(t)
fig.show()

### pivot_table()

In [32]:
# 각 도시에 대한 연도별 인구 데이터 추출
r = df1.set_index(["도시", "연도"])
r.unstack(level = "연도")

p = df1.pivot_table(index = "도시", columns = "연도", values = "인구")
p

q = df1.pivot_table(index = ["지역", "도시"], columns ="연도", values = "인구")
q

v = df1.pivot_table(index = ["지역", "도시"], values = "인구")
v

인구                         지역          
연도       2005       2010       2015 2005 2010 2015
도시                                                
부산  3512547.0  3393191.0  3448737.0  경상권  경상권  경상권
서울  9762546.0  9631482.0  9904312.0  수도권  수도권  수도권
인천        NaN   263203.0  2890451.0  NaN  수도권  수도권

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

인구
지역  도시              
경상권 부산  3.451492e+06
수도권 서울  9.766113e+06
    인천  1.576827e+06

### 타이타닉
#### 변수설명  
1. passengerId : 각 승객의 고유 번호  
2. survived : 생존 여부
    1. 0 = 사망  
    2. 1 = 생존  
3. class : 객실 등급 - 승객의 사회적, 경제적 지위  
    First    
    Second     
    Third  
4. sex : 성별  
5. age : 나이  
6. sibsp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수  
7. parch : 동반한 Parent(부모) Child(자식)의 수   
8. fare : 티켓의 요금  
9. embark_town : 승선한 항  
    1. C = Cherbourg (셰르부르)  
    2. Q = Queenstown  
    3. S = Southampton

In [36]:
df_titanic = pd.read_csv("./data/titanic_train.csv")
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [39]:
df2 = df_titanic.pivot_table(index = "Pclass", columns="Sex", values="Age")
df2

Sex,female,male
Pclass,,
1,34.611765,41.281386
2,28.722973,30.740707
3,21.750000,26.507589


In [40]:
# 선실 등급별 숙박객의 성별에 따른 생존자 수
df3 = df_titanic.pivot_table(index="Pclass", columns="Sex", values="Survived", aggfunc=["mean", "sum"])
df3

mean              sum     
Sex       female      male female male
Pclass                                
1       0.968085  0.368852     91   45
2       0.921053  0.157407     70   17
3       0.500000  0.135447     72   47

In [41]:
# 선실등급에 따른 성별에 대한 생존여부별로 나이와 티켓값의 평균과 최대값 출력
df4 = df_titanic.pivot_table(index=["Pclass", "Sex"], columns="Survived", values=["Age", "Fare"], aggfunc=["mean", "max"])
df4

mean                                      max        \
                     Age                   Fare               Age         
Survived               0          1           0           1     0     1   
Pclass Sex                                                                
1      female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
2      female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
3      female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   
       male    27.255814  22.274211   12.204469   15.579696  74.0  45.0   

                                 
                 Fare            
Survived            0         1  
Pclass Sex                       
1      female  151.55  512.3292  
       male    263.00  512.3292  
2      female   26.00   65.0000  
       male     73.50   39.0000  
3      female   69.55   31.3875  
       male     69.55   56.4958

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   z=[[1, None, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, -10, 20]],
                   x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                   y=['Morning', 'Afternoon', 'Evening'],
                   hoverongaps = False))
fig.show()